# Optimizacion de hiperparametros

In [1]:
import pandas as pd
import polars as pl
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit, StratifiedShuffleSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer

import lightgbm as lgb

import optuna
from optuna.visualization import plot_optimization_history, plot_param_importances, plot_slice, plot_contour

from time import time

import pickle

In [ ]:
# !gsutil cp /home/eanegrin/buckets/b1/datasets/competencia_03_fe_v03_undersampled.parquet /home/eanegrin/datasets/

Copying file:///home/eanegrin/buckets/b1/datasets/competencia_03_fe_v03_undersampled.parquet...
- [1 files][726.3 MiB/726.3 MiB]                                                
Operation completed over 1 objects/726.3 MiB.                                    


In [2]:
base_path = 'C:/Eugenio/Maestria/DMEyF/'
# base_path = '/home/eanegrin/buckets/b1/'

dataset_path = base_path + 'datasets/'
modelos_path = base_path + 'modelos/'
db_path = base_path + 'db/'
dataset_file = 'competencia_03_fe_v05_undersampled.parquet'

ganancia_acierto = 273000
costo_estimulo = 7000

semillas = [122219, 109279, 400391, 401537, 999961]

# data = pd.read_parquet('/home/eanegrin/datasets/' + dataset_file)
data = pd.read_parquet(dataset_path + dataset_file)

In [3]:
# el dataset undersampleado ya excluye los meses "malos" y los que tienen la clase ternaria incompleta
# nos queda excluir el mes de testing:

data = data[data['foto_mes'] != 202107]

data['foto_mes'].unique()

array([201906, 201907, 201908, 201909, 201910, 201911, 201912, 202001,
       202002, 202003, 202004, 202005, 202007, 202008, 202009, 202010,
       202011, 202012, 202101, 202102, 202103, 202104, 202105, 202106])

In [4]:
data.shape

(196883, 1278)

In [5]:
# Asignamos pesos a las clases

data['clase_peso'] = 1.0

data.loc[data['clase_ternaria'] == 'BAJA+2', 'clase_peso'] = 1.00002
data.loc[data['clase_ternaria'] == 'BAJA+1', 'clase_peso'] = 1.00001

In [6]:
X_train = data.drop(['clase_ternaria', 'clase_peso', 'clase_binaria'], axis=1)
y_train_binaria = data['clase_binaria'] # Junta a los 2 baja
w_train = data['clase_peso']

Para evaluar la calidad del modelo, crearemos nuestra propia función de evaluación que calcule la ganancia. La razón de incluir los pesos es precisamente para poder implementar esta función de evaluación de manera adecuada. Al combinar las clases *BAJA+1* y *BAJA+2* en una sola, necesitamos una forma de diferenciarlas, y es aquí donde entra en juego el *weight*. Este parámetro nos permitirá distinguir entre ambas clases al momento de evaluarlas dentro del algoritmo.


In [7]:
def lgb_gan_eval(y_pred, data):
    weight = data.get_weight()
    ganancia = np.where(weight == 1.00002, ganancia_acierto, 0) - np.where(weight < 1.00002, costo_estimulo, 0)
    ganancia = ganancia[np.argsort(y_pred)[::-1]]
    ganancia = np.cumsum(ganancia)

    return 'gan_eval', np.max(ganancia) , True

# Optimizacion

In [8]:
def objective(trial):

    num_leaves = trial.suggest_int('num_leaves', 8, 1024),
    learning_rate = trial.suggest_float('learning_rate', 0.005, 0.05),
    min_data_in_leaf = trial.suggest_int('min_data_in_leaf', 64, 8000),
    feature_fraction = trial.suggest_float('feature_fraction', 0.1, 1.0),
    bagging_fraction = trial.suggest_float('bagging_fraction', 0.1, 1.0),

    params = {
        'objective': 'binary',
        'metric': 'custom',
        'boosting_type': 'gbdt',
        'first_metric_only': True,
        'boost_from_average': True,
        'feature_pre_filter': False,
        'max_bin': 31,
        'num_leaves': num_leaves,
        'learning_rate': learning_rate,
        'min_data_in_leaf': min_data_in_leaf,
        'feature_fraction': feature_fraction,
        'bagging_fraction': bagging_fraction,
        'seed': semillas[0],
        'verbose': -1
    }
    
    train_data = lgb.Dataset(X_train,
                              label=y_train_binaria,
                              weight=w_train)
    
    # print(f"Learning Rate: {learning_rate}, Type: {type(learning_rate)}")
    
    # callbacks for early stopping
    early_stopping_cb = lgb.early_stopping(stopping_rounds=200)
        
    cv_results = lgb.cv(
        params,
        train_data,
        num_boost_round=2000,
        callbacks=[early_stopping_cb],
        feval=lgb_gan_eval,
        stratified=True,
        nfold=5,
        seed=semillas[0]
    )
    
    max_gan = max(cv_results['valid gan_eval-mean'])
    best_iter = cv_results['valid gan_eval-mean'].index(max_gan) + 1

    # Guardamos cual es la mejor iteración del modelo
    trial.set_user_attr("best_iter", best_iter)

    return max_gan * 5 # funcion objetivo, en el proximo paso le digo si quiero maximizarla o minimizarla.

In [9]:
storage_name = "sqlite:///" + db_path + "optimization_lgbm_v05.db"
study_name = "competencia3_lgbm_v05" # UPDATE

study = optuna.create_study(
    direction="maximize",
    study_name=study_name,
    storage=storage_name,
    load_if_exists=True,
)

[I 2024-12-03 00:12:26,347] A new study created in RDB with name: competencia3_lgbm_v05


In [10]:
study.optimize(objective, n_trials=100)

Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[1576]	cv_agg's valid gan_eval: 7.44439e+08 + 4.29587e+06


[I 2024-12-03 00:19:46,506] Trial 0 finished with value: 3722194000.0 and parameters: {'num_leaves': 239, 'learning_rate': 0.014066695831473366, 'min_data_in_leaf': 5900, 'feature_fraction': 0.21419666172350352, 'bagging_fraction': 0.8226409420150059}. Best is trial 0 with value: 3722194000.0.


Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[692]	cv_agg's valid gan_eval: 7.43912e+08 + 3.45969e+06


[I 2024-12-03 00:22:39,180] Trial 1 finished with value: 3719562000.0 and parameters: {'num_leaves': 897, 'learning_rate': 0.047698085458175714, 'min_data_in_leaf': 6238, 'feature_fraction': 0.8842954308609321, 'bagging_fraction': 0.7325911168654635}. Best is trial 0 with value: 3722194000.0.


Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[860]	cv_agg's valid gan_eval: 7.46704e+08 + 2.86611e+06


...

Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[622]	cv_agg's valid gan_eval: 7.46582e+08 + 3.63818e+06


[I 2024-12-03 08:43:32,139] Trial 83 finished with value: 3732911000.0 and parameters: {'num_leaves': 650, 'learning_rate': 0.01339463968618236, 'min_data_in_leaf': 590, 'feature_fraction': 0.832423476746337, 'bagging_fraction': 0.5780992732617724}. Best is trial 61 with value: 3737566000.0.


Training until validation scores don't improve for 200 rounds


[W 2024-12-03 08:49:33,062] Trial 84 failed with parameters: {'num_leaves': 520, 'learning_rate': 0.02516286097916349, 'min_data_in_leaf': 843, 'feature_fraction': 0.6374598554273491, 'bagging_fraction': 0.7157367358185518} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\eaneg\AppData\Local\Programs\Python\Python312\Lib\site-packages\optuna\study\_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\eaneg\AppData\Local\Temp\ipykernel_29252\509213872.py", line 35, in objective
    cv_results = lgb.cv(
                 ^^^^^^^
  File "c:\Users\eaneg\AppData\Local\Programs\Python\Python312\Lib\site-packages\lightgbm\engine.py", line 826, in cv
    cvfolds.update(fobj=fobj)  # type: ignore[call-arg]
    ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\eaneg\AppData\Local\Programs\Python\Python312\Lib\site-packages\lightgbm\engine.py", line 402, in handler_function
    re

KeyboardInterrupt: 

Analizamos los resultados as usual

In [11]:
optuna.visualization.plot_optimization_history(study)

In [12]:
plot_param_importances(study)

In [13]:
plot_slice(study)

In [14]:
plot_contour(study)

In [15]:
plot_contour(study, params=['num_leaves','min_data_in_leaf'])

In [16]:
study.best_trial.params

{'num_leaves': 243,
 'learning_rate': 0.016794504627976162,
 'min_data_in_leaf': 866,
 'feature_fraction': 0.7678004709395572,
 'bagging_fraction': 0.5901190076375339}

In [17]:
best_iter = study.best_trial.user_attrs["best_iter"]
best_iter

621